# **Take the top 3 active molecules and generate novel molecules** #

### 1. generate novel molecules ###

In [1]:
import pandas as pd
from padelpy import from_smiles
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

In [2]:
# QSAR Regression Model
def calculate_pec50(descriptors):
    return (
        -87.3 + 0.33 * descriptors["MDEC-33"]
        - 5.14 * descriptors["VE1_Dzp"]
        - 0.16 * descriptors["ATSC6e"]
        - 3.49 * descriptors["minaaN"]
        + 28.24 * descriptors["SpMax4_Bhm"]
        + 0.28 * descriptors["nAtomLAC"]
        - 0.08 * descriptors["VE3_Dzs"]
    )

In [4]:
# Global Enumeration: Atom-based and Reaction-based substitutions
def enumerate_molecules(smiles):
    mol = Chem.MolFromSmiles(smiles)
    variants = set()

    # Define common replacements for atom-based enumeration
    replacements = {
        "F": ["Cl", "Br", "I", "H"],
        "Cl": ["F", "Br", "I", "H"],
        "Br": ["F", "Cl", "I", "H"],
        "OH": ["NH2", "OCH3", "SH"],
        "CH3": ["H", "C2H5", "OH", "NH2"],
    }

    # Atom-based substitutions
    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        if symbol in replacements:
            for replacement in replacements[symbol]:
                mol_copy = Chem.RWMol(mol)
                idx = atom.GetIdx()
                mol_copy.GetAtomWithIdx(idx).SetAtomicNum(
                    Chem.GetPeriodicTable().GetAtomicNumber(replacement)
                )
                variant_smiles = Chem.MolToSmiles(mol_copy, isomericSmiles=True)
                variants.add(variant_smiles)

    # Reaction-based substitutions (functional group enumeration)
    reaction_smarts = [
        "[C:1]F>>[C:1]Cl",
        "[C:1]F>>[C:1]Br",
        "[C:1]F>>[C:1]I",
        "[O:1]>>[N:1]",
    ]
    for rxn_smarts in reaction_smarts:
        rxn = AllChem.ReactionFromSmarts(rxn_smarts)
        products = rxn.RunReactants((mol,))
        for product_set in products:
            for product in product_set:
                product_smiles = Chem.MolToSmiles(product, isomericSmiles=True)
                variants.add(product_smiles)

    # Return all unique enumerations
    return list(variants)

In [5]:
# Calculate Leverage
def calculate_leverage(x, X):
    X = np.array(X)
    x = np.array(x)
    XTX_inv = np.linalg.inv(np.dot(X.T, X))
    h = np.dot(np.dot(x.T, XTX_inv), x)
    return h

In [6]:
# Main function to process leads
def process_leads(leads, training_data, cutoff):
    # Extract training descriptors matrix (X) from the training data
    descriptor_columns = training_data.columns[1:-1]  # All except Molecule and pEC50
    X = training_data[descriptor_columns].values

    results = []
    for lead_smiles in leads:
        enumerated_molecules = enumerate_molecules(lead_smiles)
        for mol_smiles in enumerated_molecules:
            try:
                # Calculate descriptors using PadelPy
                descriptors = from_smiles(mol_smiles)
                relevant_descriptors = {name: float(descriptors[name]) for name in descriptor_columns}

                # Predict pEC50
                pec50 = calculate_pec50(relevant_descriptors)

                # Calculate leverage
                x = [relevant_descriptors[name] for name in descriptor_columns]
                leverage = calculate_leverage(x, X)

                # Add remark based on leverage
                remark = "Outlier" if leverage > cutoff else "Inside"

                # Append results
                results.append({
                    "SMILES": mol_smiles,
                    **relevant_descriptors,
                    "pEC50": pec50,
                    "Leverage": leverage,
                    "Remark": remark
                })
            except Exception as e:
                print(f"Error processing molecule {mol_smiles}: {e}")

    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv("new_molecules.csv", index=False)

In [10]:
# Load training data
training_data = pd.read_csv("train_data.csv")

In [11]:
# Input: Lead SMILES
lead_smiles = [
    "O=C(Nc1nc(-c2cccc(C(F)(F)I)c2F)cs1)c1ccc(Nc2cc(N3CC(O)C3)ncn2)cc1",
    "O=C(Nc1nc(-c2cccc(C(F)(F)Br)c2F)cs1)c1ccc(Nc2cc(N3CC(O)C3)ncn2)cc1",
    "O=C(Nc1nc(-c2cccc(C(F)(F)F)c2Br)cs1)c1ccc(Nc2cc(N3CC(O)C3)ncn2)cc1"
]

In [12]:
# Set leverage cutoff value
cutoff = 0.37

# Process leads and save results
process_leads(lead_smiles, training_data, cutoff)

In [15]:
df = pd.read_csv("new_molecules.csv")
df.head()

,SMILES,MDEC-33,VE1_Dzp,ATSC6e,minaaN,SpMax4_Bhm,nAtomLAC,VE3_Dzs,pEC50,Leverage,Remark
0,O=C(Nc1nc(-c2cccc(C(F)(F)I)c2Br)cs1)c1ccc(Nc2c...,9.897105,0.228187,-0.243374,4.259471,3.877958,0.0,-8.342843,10.147511,0.118368,Inside
1,[H]c1c(-c2csc(NC(=O)c3ccc(Nc4cc(N5CC(O)C5)ncn4...,7.906650,0.164997,-0.470203,4.251473,3.794571,0.0,-8.187258,7.512366,0.122494,Inside
2,O=C(Nc1nc(-c2cccc(C(F)(F)I)c2Cl)cs1)c1ccc(Nc2c...,9.897105,0.224944,-0.147025,4.258499,3.802752,0.0,-8.062742,8.005923,0.147141,Inside
3,O=C(Nc1nc(-c2cccc(C(F)(F)I)c2I)cs1)c1ccc(Nc2cc...,9.897105,0.231670,-0.404269,4.259609,3.880084,0.0,-8.397966,10.219318,0.123700,Inside
4,O=C(Nc1nc(-c2cccc(C(F)(Br)I)c2F)cs1)c1ccc(Nc2c...,9.897105,0.232414,0.011658,4.254685,3.884043,0.0,-9.318228,10.351553,0.117986,Inside


### 2. Extract molecules with higher activity and inside the applicability domain ###

In [16]:
# Define the cutoff value
cutoff_value = 8.0  

# Filter the DataFrame
filtered_df = df[(df['pEC50'] > cutoff_value) & (df['Remark'] == 'Inside')]

# Select relevant columns and sort by pEC50 in descending order
result_df = filtered_df[['SMILES', 'pEC50']].sort_values(by='pEC50', ascending=False)

# Display the resulting DataFrame
print(result_df)

                                               SMILES      pEC50
12  O=C(Nc1nc(-c2cccc(C(F)(Br)Br)c2F)cs1)c1ccc(Nc2...  10.354281
4   O=C(Nc1nc(-c2cccc(C(F)(Br)I)c2F)cs1)c1ccc(Nc2c...  10.351553
16  O=C(Nc1nc(-c2cccc(C(F)(Br)I)c2F)cs1)c1ccc(Nc2c...  10.351553
7   O=C(Nc1nc(-c2cccc(C(F)(I)I)c2F)cs1)c1ccc(Nc2cc...  10.350985
3   O=C(Nc1nc(-c2cccc(C(F)(F)I)c2I)cs1)c1ccc(Nc2cc...  10.219318
20  O=C(Nc1nc(-c2cccc(C(F)(F)Br)c2I)cs1)c1ccc(Nc2c...  10.217241
0   O=C(Nc1nc(-c2cccc(C(F)(F)I)c2Br)cs1)c1ccc(Nc2c...  10.147511
29  O=C(Nc1nc(-c2cccc(C(F)(F)I)c2Br)cs1)c1ccc(Nc2c...  10.147511
21  O=C(Nc1nc(-c2cccc(C(F)(F)Br)c2Br)cs1)c1ccc(Nc2...  10.143638
32  O=C(Nc1nc(-c2cccc(C(F)(F)Br)c2Br)cs1)c1ccc(Nc2...  10.143638
28  O=C(Nc1nc(-c2cccc(C(F)(F)Cl)c2Br)cs1)c1ccc(Nc2...   8.903441
17  [H]C(F)(Br)c1cccc(-c2csc(NC(=O)c3ccc(Nc4cc(N5C...   8.529132
8   [H]C(F)(I)c1cccc(-c2csc(NC(=O)c3ccc(Nc4cc(N5CC...   8.528930
30  [H]C(F)(F)c1cccc(-c2csc(NC(=O)c3ccc(Nc4cc(N5CC...   8.423636
18  O=C(Nc1nc(-c2cccc(C(F

In [18]:
result_df.to_csv("new_molecules_filtered.csv", index=False)

In [19]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 12 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SMILES  16 non-null     object 
 1   pEC50   16 non-null     float64
dtypes: float64(1), object(1)
memory usage: 384.0+ bytes


### 3. Check if the SMILES are valid

In [20]:
from rdkit import Chem

# Function to check if a SMILES string is valid
def is_valid_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False

# Add a column to result_df indicating whether the SMILES is valid
result_df['Is_Valid_SMILES'] = result_df['SMILES'].apply(is_valid_smiles)

# Display the updated DataFrame
print(result_df)

# Optionally, filter out invalid SMILES
valid_smiles_df = result_df[result_df['Is_Valid_SMILES'] == True]

# Save the filtered DataFrame if needed
valid_smiles_df.to_csv('valid_smiles.csv', index=False)

                                               SMILES      pEC50   
12  O=C(Nc1nc(-c2cccc(C(F)(Br)Br)c2F)cs1)c1ccc(Nc2...  10.354281  \
4   O=C(Nc1nc(-c2cccc(C(F)(Br)I)c2F)cs1)c1ccc(Nc2c...  10.351553   
16  O=C(Nc1nc(-c2cccc(C(F)(Br)I)c2F)cs1)c1ccc(Nc2c...  10.351553   
7   O=C(Nc1nc(-c2cccc(C(F)(I)I)c2F)cs1)c1ccc(Nc2cc...  10.350985   
3   O=C(Nc1nc(-c2cccc(C(F)(F)I)c2I)cs1)c1ccc(Nc2cc...  10.219318   
20  O=C(Nc1nc(-c2cccc(C(F)(F)Br)c2I)cs1)c1ccc(Nc2c...  10.217241   
0   O=C(Nc1nc(-c2cccc(C(F)(F)I)c2Br)cs1)c1ccc(Nc2c...  10.147511   
29  O=C(Nc1nc(-c2cccc(C(F)(F)I)c2Br)cs1)c1ccc(Nc2c...  10.147511   
21  O=C(Nc1nc(-c2cccc(C(F)(F)Br)c2Br)cs1)c1ccc(Nc2...  10.143638   
32  O=C(Nc1nc(-c2cccc(C(F)(F)Br)c2Br)cs1)c1ccc(Nc2...  10.143638   
28  O=C(Nc1nc(-c2cccc(C(F)(F)Cl)c2Br)cs1)c1ccc(Nc2...   8.903441   
17  [H]C(F)(Br)c1cccc(-c2csc(NC(=O)c3ccc(Nc4cc(N5C...   8.529132   
8   [H]C(F)(I)c1cccc(-c2csc(NC(=O)c3ccc(Nc4cc(N5CC...   8.528930   
30  [H]C(F)(F)c1cccc(-c2csc(NC(=O)c3ccc(Nc4cc(N5